<a href="https://colab.research.google.com/github/Lucassilva00/voice_qa_assistant/blob/main/notebooks/voice_qa_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Define a linguagem a ser usada pelo whisper e gTTS
language = 'pt'

#1. Gravação de Áudio Com Python

In [5]:
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))

  js_result = output.eval_js('record(%s)' % (sec * 1000))

  audio = b64decode(js_result.split(',')[1])

  #Salva o áudio
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)

  return f'/content/{file_name}'

print('Gravando...\n')
record_file = record()

display(Audio(record_file, autoplay = False))


Gravando...



<IPython.core.display.Javascript object>

#2. Reconhecimento de Fala com Whisper

In [6]:
!pip install git+https://github.com/openai/whisper.git -q

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-1mhaggn4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-1mhaggn4
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
import whisper

#Escolha do modelo
model = whisper.load_model("small")

#Transcreve o áudio e mostra o resultado
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

 Em poucas linhas me diga para que serve o Google Colab.


#3. Integração com a API do Google Gemini


In [13]:
!pip install -q -U google-genai

In [15]:
from google import genai
from google.colab import userdata

#Na aba secret cria-se uma chave secreta e substitua secretName pelo nome de sua chave
import os
os.environ["GEMINI_API_KEY"] = userdata.get("secretName")

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=transcription
)

print(response.text)

 Em poucas linhas me diga para que serve o Google Colab.
O Google Colab serve para **executar código Python diretamente no navegador**, usando a estrutura de **notebooks Jupyter**, sem a necessidade de configurar um ambiente local.

É especialmente útil para **desenvolvimento e experimentação em machine learning e data science**, pois oferece **acesso gratuito a GPUs e TPUs** para processamento.


#4. Sintetizando a Resposta do Gemini com Voz (gTTS)

In [1]:
!pip install gTTS

In [12]:
from gtts import gTTS

gtts_object = gTTS(text=response.text, lang=language, slow=False)

response_audio = "/content/response_audio.mp3"
gtts_object.save(response_audio)

display(Audio(response_audio, autoplay=False))